In [1]:
import obspy as obs
from glob import glob
import numpy as np
import sys
import os
from obspy import read
import matplotlib.pyplot as plt
np.set_printoptions(suppress = True)
from IPython.display import clear_output

In [2]:
cc_matrix = np.zeros([29412, 5+1201]) # length of traces is 1201 each... and first 5 values will serve as headers
a = 0

for i in np.arange(172):
    for j in np.arange(172):
        if i != j:
            cc_matrix[a,0] = a+1
            cc_matrix[a,1] = i+1
            cc_matrix[a,2] = j+1
            a +=1
        
        else:
            continue

In [3]:
corlist = glob('MAY2014/2014.05.month/DATA/correlations/noisy/*.SAC')

In [4]:
freqmin=0.55
freqmax=0.95
corners=3

for file in corlist:
    a = int(file.split('\\')[1].split('.')[1])
    b = int(file.split('\\')[1].split('.')[5])
    #print(a)
    #print(b)
    
    # find the index that corresponds to the receiver pair
    index = np.where((cc_matrix[:,1] == a) & (cc_matrix[:,2] == b))[0][0]
    #print(index)
    index_r = np.where((cc_matrix[:,1] == b) & (cc_matrix[:,2] == a))[0][0]
    #print(index_r)
    
    # read in the cc data and filter
    cc_data = read(file)
    cc_data.taper(max_percentage = 0.02, type = 'cosine')
    cc_data.filter(type='bandpass',freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
    
    # include some additional header information on distance and azimuth, and then include data
    cc_matrix[index,3] = cc_data[0].stats['sac']['dist']
    cc_matrix[index,4] = cc_data[0].stats['sac']['az']
    cc_matrix[index,5:]   = cc_data[0].data
    
    # do the same for the reverse pair - redundant ones
    cc_matrix[index_r,3] = cc_data[0].stats['sac']['dist']
    if cc_data[0].stats['sac']['az'] + 180 > 360:
        cc_matrix[index_r,4] = cc_data[0].stats['sac']['az'] - 180
    else:
        cc_matrix[index_r,4] = cc_data[0].stats['sac']['az'] + 180
    cc_matrix[index_r,5:] = np.flip(cc_data[0].data)

In [5]:
import numpy as np
import scipy.io

scipy.io.savemat('2014.05.month_0.55.to.0.95_3_redundant.mat', dict(headers=cc_matrix[:,0:5], cc_matrix=cc_matrix[:,5:], fold=10351))